In [8]:
# import splinter and BeatifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from quant_up import user, password
import time 
import datetime
import pandas as pd

In [9]:
### Navigate to quantum online,input credentials and navigate to securities for redemption 

In [10]:
# confirm todays date
datetime.datetime.today().strftime("%Y-%m-%d")

'2023-03-16'

In [11]:
# redemptions page reoutes to login, 
browser = Browser('chrome')

In [12]:
# Visit quantum https://www.quantumonline.com/stklist.cfm?type=Redemptions&RequestTimeout=60
url = 'https://www.quantumonline.com/stklist.cfm?type=Redemptions&RequestTimeout=60'
browser.visit(url)

In [13]:
# path to login click
browser.find_by_xpath('/html/body/table[2]/tbody/tr/td[2]/font[2]/p[1]/b/a').click()

In [14]:
# path to login click
browser.find_by_xpath('/html/body/table/tbody/tr[2]/td/table/tbody/tr[3]/td[9]/a/img').click()


In [15]:
#input user and pass

browser.find_by_xpath('/html/body/table[2]/tbody/tr/td[2]/font/font/blockquote/center/font/table/tbody/tr[2]/td/input').fill(user)
                      
browser.find_by_xpath('/html/body/table[2]/tbody/tr/td[2]/font/font/blockquote/center/font/table/tbody/tr[3]/td/input').fill(password)

# click
browser.find_by_xpath('/html/body/table[2]/tbody/tr/td[2]/font/font/blockquote/center/font/table/tbody/tr[4]/td/input').click()


In [16]:
#path to special lists
browser.find_by_xpath('/html/body/table[1]/tbody/tr[2]/td/table/tbody/tr[3]/td[6]/a/img').click()

In [17]:
#path to securities for redemptions
browser.find_by_xpath('/html/body/font/table/tbody/tr/td[2]/font[2]/p[2]/b[2]/a').click()
time.sleep(5)

In [18]:
### SCRAPE

In [19]:
#scrape
html=browser.html
redemption_soup = soup(html, 'html.parser')

In [20]:
# Create empty lists for the table headers and rows
header_list = []
row_list = []

# Find all table rows
tables = redemption_soup.find_all('tr')

# Loop through the table rows
for i, row in enumerate(tables):
    # Find all table cells (td) and table headers (th) within the row
    cells = row.find_all(['td', 'th'])
    # If this is the first row, treat it as the header row
    if i == 0:
        # Extract the text from each cell and append to the header list
        header_list = [cell.text.strip() for cell in cells]
    else:
        # Extract the text from each cell and append to the row list
        row_data = [cell.text.strip() for cell in cells]
        row_list.append(row_data)

# Print the header and row lists
print(header_list)
print(row_list)


['Quick Search\n\n\n\n\nby Ticker Symbol\nby CUSIP Number\nSymbol Lookup', 'Quick Search', '', 'by Ticker Symbol\nby CUSIP Number\nSymbol Lookup', '', 'Welcome, \xa0\xa0\xa0Sarah Peterson \n\t\t\t,\xa0\xa0\xa0  Non-Contributor (Please click here to make a contribution) \n\t\t\t\n\t\t\tDays of use: 7', '348304', '']
[['Quick Search'], [''], ['by Ticker Symbol\nby CUSIP Number\nSymbol Lookup'], [''], [''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', ''], ['', ''], ['', '', '', '', '', '', '', '', '', '', ''], ['', ''], ['', ''], ['Securities Called for Redemption'], ["A list of 2690 preferred and other income securities that have been called for redemption since April 2002.  The Original Call Date column shows the date originally specified in the security's prospectus as the redemption (call) date while the Actual Call Date column shows the date that the security was actuall

In [45]:
# convert scrape to DataFrame; goal is to to have the td in 4 columns named "Symbol", "Security", "Original Call Date",
## "Actual Call Date"
# Find the starting index for 'Symbol' in row_list
start_index = None
for i, row in enumerate(row_list):
    if 'Symbol' in row:
        start_index = i
        break

# Remove all rows before the 'Symbol' row (including the header row)
data = row_list[start_index+1:]

# Create a pandas DataFrame from the remaining rows
SFR_df = pd.DataFrame(data)
#drop rows 4:13
SFR_df = SFR_df.drop(SFR_df.columns[4:14], axis=1)
#rename columns
SFR_df.columns=['Symbol', 'Security', 'Original Call Date', 'Actual Date Called']
SFR_df.head()


,Symbol,Security,Original Call Date,Actual Date Called
0,ALL-G,"Allstate Corp., 5.625% Dep Shares Non-Cumulati...",4/15/23,4/17/23
1,RJF-A,"Raymond James Financial, 6.75% Dep Sh Fix/Floa...",4/01/23,4/03/23
2,PSBZP,"PS Business Parks, 4.875% Dep Shares Cumulativ...",11/04/24,3/29/23
3,PSBYP,"PS Business Parks, 5.20% Dep Shares Cumulative...",12/07/22,3/29/23
4,PSBXP,"PS Business Parks, 5.25% Dep Shares Cumulative...",9/21/22,3/29/23


In [46]:
## Data cleaning: drop unneccessary rows 
SFR_df.drop(SFR_df.tail(7).index, inplace = True)
SFR_df.tail()

,Symbol,Security,Original Call Date,Actual Date Called
99,AGO-E*,"Assured Guaranty Municipal Holdings Inc., 6.25...",11/26/07,9/27/21
100,GGO-A*,"Gabelli Go Anywhere Trust, Series A Cumul Putt...",8/26/21,9/26/21
101,CBB-B*,"Cincinnati Bell Inc., 6 3/4% Dep Shares Cumul ...",4/05/00,9/22/21
102,WFC-X*,"Wells Fargo & Co., 5.50% Dep Shares Non-Cumul ...",9/15/21,9/15/21
103,Symbol,Security,OriginalCall Date,ActualDate Called


In [47]:
# define the regular expression to match the order of strings
pattern = 'Symbol.*Security.*OriginalCall Date.*ActualDate Called'

# filter the dataframe to drop the rows that match the pattern
SFR_df = SFR_df[~SFR_df.astype(str).apply(lambda x: x.str.contains(pattern)).any(axis=1)]

# display the updated dataframe
SFR_df.tail()

,Symbol,Security,Original Call Date,Actual Date Called
99,AGO-E*,"Assured Guaranty Municipal Holdings Inc., 6.25...",11/26/07,9/27/21
100,GGO-A*,"Gabelli Go Anywhere Trust, Series A Cumul Putt...",8/26/21,9/26/21
101,CBB-B*,"Cincinnati Bell Inc., 6 3/4% Dep Shares Cumul ...",4/05/00,9/22/21
102,WFC-X*,"Wells Fargo & Co., 5.50% Dep Shares Non-Cumul ...",9/15/21,9/15/21
103,Symbol,Security,OriginalCall Date,ActualDate Called


In [48]:
browser.quit()